In [1]:
import time
import clingo

In [2]:
def solve_asp(program, num_answer_sets=1):
    control = clingo.Control(arguments=["--project"])
    control.add("base", [], program)
    control.ground([("base", [])])
    control.configuration.solve.models = num_answer_sets
    answer = control.solve()
    if answer.satisfiable == False:
        print("No answer sets")

In [3]:
def solve_backtracking(board):

    def solve_h():
        def possible_placement(r, c, n):
            if (any(board[r][i]==n for i in range(9)) or
                any(board[i][c]==n for i in range(9)) or
                any(board[i][j]==n for i in range(3*(r//3), 3*(r//3)+3) 
                                for j in range(3*(c//3), 3*(c//3)+3))):
                return False
            return True

        for r in range(9):
            for c in range(9):
                if board[r][c] == 0:
                    for n in range(1, 10):
                        if possible_placement(r, c, n):
                            board[r][c] = n
                            if solve_h():
                                return True
                            board[r][c] = 0  # Backtrack
                    return False
        return True

    return solve_h()

In [4]:
puzzle = [[0, 4, 6, 1, 0, 2, 0, 7, 3], [0, 0, 8, 9, 0, 3, 5, 0, 0], [1, 0, 0, 6, 8, 7, 0, 2, 4], [5, 7, 0, 2, 0, 4, 1, 6, 8], [3, 6, 0, 0, 0, 0, 0, 5, 9], [4, 8, 1, 0, 9, 0, 7, 0, 2], [0, 0, 0, 3, 6, 5, 0, 9, 0], [2, 3, 0, 0, 7, 0, 0, 8, 1], [6, 9, 7, 8, 0, 1, 0, 4, 0]]

In [5]:
asp_program = str()
count = 0
for idx, row in enumerate(puzzle):
    for id, val in enumerate(row):
        asp_program += f"grid({idx},{id},{val}).\n"
        if val:
            asp_program += f"solution({idx},{id},{val}).\n"
asp_program += f"""
1{{solution(X, Y, Num) : 0 < Num < 10}}1 :- grid(X, Y, 0).
:- solution(X,Y1,Val), solution(X,Y2,Val), Y1 != Y2.
:- solution(X1,Y,Val), solution(X2,Y,Val), X1 != X2.


:- solution(X1,Y1,Val), solution(X2,Y2,Val), 
   block(X1, Y1, B), block(X2, Y2, B), X1 != X2.
:- solution(X1,Y1,Val), solution(X2,Y2,Val), 
   block(X1, Y1, B), block(X2, Y2, B), Y1 != Y2.

block(X, Y, B) :- X >= B*3, X < (B+1)*3, Y >= B*3, Y < (B+1)*3, B = 0..2.

#show solution/3.
"""

In [6]:
start_time = time.time()
for _ in range(10_000):
   catch = solve_asp(asp_program)
print(f"ASP time: {time.time() - start_time :.2f}s")

start_time = time.time()
for _ in range(10_000):
   catch = solve_asp(asp_program)
print(f"Python backtracking time: {time.time() - start_time :.2f}s")

ASP time: 31.61s
Python backtracking time: 30.71s
